In [1]:
# Import libraries used for collaborative filtering

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Flatten, Dense, Input, Concatenate
from tensorflow.keras.models import Model


2024-06-20 02:37:00.265096: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-20 02:37:00.265163: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-20 02:37:00.302160: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-20 02:37:00.373813: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Load the data
df = pd.read_csv('final-dataset/review_dataset.csv')

# Encode id and user id
user_enc = LabelEncoder()
df['user_id_encoded'] = user_enc.fit_transform(df['user_id'].values)

place_enc = LabelEncoder()
df['place_id_encoded'] = place_enc.fit_transform(df['id'].values)

n_users = df['user_id_encoded'].nunique()
n_places = df['place_id_encoded'].nunique()

print(f'Number of users: {n_users} and number of places: {n_places}')
df.head(10)

Number of users: 6103 and number of places: 6178


,id,types,review_number,review,user_id,sentiment-score,user_id_encoded,place_id_encoded
0,ChIJYcGr7GSb0S0RckePBrCWikw,"hotel, lodging",review 1,"It has quite small room, and the hallway is qu...",user_2573,0.506250,1732,3451
1,ChIJZbWX6Aia0S0R0tM3h1RZ1h8,"indonesian_restaurant, restaurant, food",review 1,"Surprisingly, a really good warung that’s hidd...",user_2062,0.606250,1170,3546
2,ChIJYyHbhgia0S0RzdjNXLmcf54,"tourist_attraction, restaurant, food",review 1,"Only had a fleeting visit here, came by coach,...",user_2348,0.491667,1484,3468
3,ChIJ6zf9LJCb0S0RFv3BdLl61ZY,"coffee_shop, cafe, food, store",review 1,"One word, underrated! How come place like this...",user_1448,0.672338,494,791
4,ChIJxaITmQia0S0RyrbukE8vsJU,"tourist_attraction, place_of_worship",review 1,"This temple is located in Singaraja, located i...",user_4426,0.569762,3759,5949
5,ChIJ63FmGgaa0S0RWD5dfwhjGHQ,"indonesian_restaurant, restaurant, food",review 1,"We came here for dinner, food was good, i like...",user_1786,0.593056,867,713
6,ChIJAQAA5Aia0S0RBL27x0I5sHk,"indonesian_restaurant, restaurant, food",review 1,"They have 3 menus. Chicken, beef, or mixed. Th...",user_4604,0.433685,3955,1136
7,ChIJA7v-3Qia0S0RdP8U7AQZBHE,"indonesian_restaurant, restaurant, food",review 1,"Did not meet my expectation...dirty table, not...",user_2813,0.400000,1991,1127
8,ChIJnSg1VAia0S0R3r6ej3XRQ50,"breakfast_restaurant, indonesian_restaurant, r...",review 1,Super yum rawon (beef stew with spices)!\r\nDe...,user_4955,0.735417,4339,4974
9,ChIJL_GkVA-a0S0R6UJc7dfTH4A,"restaurant, coffee_shop, cafe, bakery, wholesa...",review 1,A Singaraga Gem...\r\nHighly recommended...\r\...,user_2987,0.654189,2182,2175


In [3]:
# Adjust if neccessary
embedding_size = 32

# User input
user_input = Input(shape=(1,), name='user_input')
user_embedding = Embedding(n_users, embedding_size, name='user_embedding')(user_input)
user_vec = Flatten(name='user_flatten')(user_embedding)

# Place input
place_input = Input(shape=(1,), name='place_input')
place_embedding = Embedding(n_places, embedding_size, name='place_embedding')(place_input)
place_vec = Flatten(name='place_flatten')(place_embedding)

# Concatenate user and place vectors
concat = Concatenate()([user_vec, place_vec])

# Add dense layers
dense_1 = Dense(128, activation='relu')(concat)
dense_2 = Dense(64, activation='relu')(dense_1)
output = Dense(1, activation='linear')(dense_2)

# Build the model
model = Model(inputs=[user_input, place_input], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Display the model summary
model.summary()

2024-06-20 02:37:02.677896: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 user_input (InputLayer)     [(None, 1)]                  0         []                            
                                                                                                  
 place_input (InputLayer)    [(None, 1)]                  0         []                            
                                                                                                  
 user_embedding (Embedding)  (None, 1, 32)                195296    ['user_input[0][0]']          
                                                                                                  
 place_embedding (Embedding  (None, 1, 32)                197696    ['place_input[0][0]']         
 )                                                                                            

2024-06-20 02:37:02.724398: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-20 02:37:02.724467: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-20 02:37:02.727133: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-20 02:37:02.727175: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-20 02:37:02.727205: I external/local_xla/xla/stream_executor

In [4]:
# Train the model
history = model.fit(
    [df['user_id_encoded'], df['place_id_encoded']],
    df['sentiment-score'],
    epochs=10
)


Epoch 1/10


2024-06-20 02:37:03.701896: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-06-20 02:37:04.022978: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f136fd63530 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-06-20 02:37:04.023019: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2024-06-20 02:37:04.033592: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-06-20 02:37:04.058821: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
I0000 00:00:1718825824.126933    2589 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


966/966 [==============================] - 7s 6ms/step - loss: 0.0209
Epoch 2/10
966/966 [==============================] - 5s 5ms/step - loss: 0.0117
Epoch 3/10
966/966 [==============================] - 5s 5ms/step - loss: 0.0078
Epoch 4/10
966/966 [==============================] - 5s 5ms/step - loss: 0.0042
Epoch 5/10
966/966 [==============================] - 5s 5ms/step - loss: 0.0026
Epoch 6/10
966/966 [==============================] - 5s 5ms/step - loss: 0.0019
Epoch 7/10
966/966 [==============================] - 5s 5ms/step - loss: 0.0016
Epoch 8/10
966/966 [==============================] - 5s 5ms/step - loss: 0.0015
Epoch 9/10
966/966 [==============================] - 5s 5ms/step - loss: 0.0014
Epoch 10/10
966/966 [==============================] - 5s 5ms/step - loss: 0.0012


In [5]:
loss = model.evaluate([df['user_id_encoded'], df['place_id_encoded']], df['sentiment-score'])
print(f"Model Loss: {loss}")


966/966 [==============================] - 2s 2ms/step - loss: 0.0012
Model Loss: 0.00120551697909832


In [6]:
model.save('collab_model.keras')

In [8]:
# Test cell

# Function to recommend places for a given user
def recommend_places(user_id, model, df, user_encoder, place_encoder, top_n=5):
    user_idx = user_encoder.transform([user_id])[0]
    all_place_ids = df['place_id_encoded'].unique()
    user_array = np.array([user_idx] * len(all_place_ids))
    
    predictions = model.predict([user_array, all_place_ids])
    
    place_indices = predictions.flatten().argsort()[-top_n:][::-1]
    recommended_places = place_encoder.inverse_transform(place_indices)
    
    return recommended_places

# Get recommendations for a specific user
user_id = "user_1069"
recommended_places = recommend_places(user_id, model, df, user_enc, place_enc, top_n=10)
print(f"Recommended places for user {user_id}: {recommended_places}")

places = pd.read_csv('combined-dataset/combined_datasetV2.csv')
places = places[places['id'].isin(recommended_places)]  # Filter places
places


ValueError: y contains previously unseen labels: '213214'